In [ ]:
import pandas as pd

# Cargar el archivo CSV
ruta = "usuarios_steam_detallado.csv"
df = pd.read_csv(ruta)

# Mostrar información general del DataFrame
print("📌 Información del DataFrame:")
print(df.info())
print("\n🔍 Primeras filas:")
print(df.head())
print("\n🧱 Columnas:")
print(df.columns.tolist())

TIEMPO_MIN_JUGADO = 60
TIEMPO_ULTIMAS_2_SEMANAS = 20
MIN_USUARIOS_POR_JUEGO =100
MIN_JUEGOS_POR_USUARIO = 3


📌 Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8672618 entries, 0 to 8672617
Data columns (total 9 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   steam_id               int64 
 1   appid                  int64 
 2   name                   object
 3   playtime_forever       int64 
 4   playtime_2weeks        int64 
 5   img_icon_url           object
 6   has_stats              bool  
 7   has_leaderboards       bool  
 8   content_descriptorids  object
dtypes: bool(2), int64(4), object(3)
memory usage: 479.7+ MB
None

🔍 Primeras filas:
            steam_id  appid                       name  playtime_forever  \
0  76561198235790210     50  Half-Life: Opposing Force                44   
1  76561198235790210     70                  Half-Life               746   
2  76561198235790210    130      Half-Life: Blue Shift                 0   
3  76561198235790210    220                Half-Life 2               767   
4  765611

In [201]:
import pandas as pd


# Eliminar duplicados dejando solo appid y nombre
juegos_unicos = df[["appid", "name"]].drop_duplicates()

# Guardar el resultado en un nuevo CSV
juegos_unicos.to_csv("nombres_juegos.csv", index=False)

print(f"✅ Dataset generado con {len(juegos_unicos)} juegos únicos.")


✅ Dataset generado con 44921 juegos únicos.


In [202]:
# Leer el archivo de usuarios
appids_a_quitar = [1366800,629520,438100,431960,2507950,1281930,714010]  # Puedes poner los que necesites

# Filtrar el DataFrame eliminando los appids indicados
df = df[~df['appid'].isin(appids_a_quitar)]
# Filtrar el DataFrame para conservar solo los juegos jugados
# df_filtrado = df[df["playtime_forever"] > TIEMPO_MIN_JUGADO].copy()
df_filtrado = df[df["playtime_2weeks"] > TIEMPO_ULTIMAS_2_SEMANAS].copy()

# Mostrar cuántas filas se han eliminado y el nuevo tamaño
filas_eliminadas = len(df) - len(df_filtrado)
print(f"🗑️ Filas eliminadas: {filas_eliminadas}")
print(f"✅ Tamaño del DataFrame filtrado: {df_filtrado.shape}")


🗑️ Filas eliminadas: 8593144
✅ Tamaño del DataFrame filtrado: (40391, 9)


In [203]:
usuarios_unicos = df_filtrado['steam_id'].nunique()
juegos_unicos = df_filtrado['appid'].nunique()

print(f"👤 Usuarios únicos: {usuarios_unicos}")
print(f"🎮 Juegos únicos: {juegos_unicos}")


👤 Usuarios únicos: 15695
🎮 Juegos únicos: 4571


In [204]:
# 1. Eliminar juegos jugados por menos de 10 usuarios
juegos_con_10_usuarios = df_filtrado['appid'].value_counts()
juegos_validos = juegos_con_10_usuarios[juegos_con_10_usuarios >= MIN_USUARIOS_POR_JUEGO].index
df_filtrado = df_filtrado[df_filtrado['appid'].isin(juegos_validos)]

# 2. Eliminar usuarios con menos de 5 juegos
usuarios_con_5_juegos = df_filtrado['steam_id'].value_counts()
usuarios_validos = usuarios_con_5_juegos[usuarios_con_5_juegos >= MIN_JUEGOS_POR_USUARIO].index
df_filtrado = df_filtrado[df_filtrado['steam_id'].isin(usuarios_validos)]

# Mostrar nuevo tamaño
print(f"📉 Tamaño del DataFrame tras limpieza adicional: {df_filtrado.shape}")


📉 Tamaño del DataFrame tras limpieza adicional: (5785, 9)


In [205]:
# Creamos la matriz usuario-juego con playtime_forever como valor
from scipy.sparse import coo_matrix
import numpy as np

# Codificar usuarios y juegos como índices
usuario_idx = {u: i for i, u in enumerate(df_filtrado['steam_id'].unique())}
juego_idx = {j: i for i, j in enumerate(df_filtrado['appid'].unique())}

# Crear listas de índices y valores
user_ids = df_filtrado['steam_id'].map(usuario_idx)
item_ids = df_filtrado['appid'].map(juego_idx)
values = df_filtrado['playtime_forever'].astype(np.float32)

# Crear matriz dispersa
matriz_interacciones = coo_matrix((values, (user_ids, item_ids)))

# Mostrar dimensiones
print(f"🧩 Matriz de interacciones creada con forma: {matriz_interacciones.shape}")


🧩 Matriz de interacciones creada con forma: (1643, 65)


In [206]:
from collections import defaultdict
import numpy as np

# === Paso 1: Calcular popularidad global por appid ===
popularidad = df_filtrado['appid'].value_counts(normalize=True).to_dict()

# Normalizar la popularidad al rango [0.5, 1.5]
min_pop, max_pop = min(popularidad.values()), max(popularidad.values())
rango = max_pop - min_pop if max_pop != min_pop else 1
popularidad_normalizada = {
    str(appid): 0.5 + (valor - min_pop) / rango
    for appid, valor in popularidad.items()
}

# === Paso 2: Crear secuencias ponderadas con popularidad y recencia ===
user_sequences_weighted = []

for user_id, group in df_filtrado.groupby('steam_id'):
    juegos = []
    for _, row in group.iterrows():
        appid = str(row['appid'])  # Word2Vec usa strings
        playtime = row['playtime_forever']
        playtime_2w = row['playtime_2weeks']

        # Base: peso por tiempo total jugado (máx 100)
        base_peso = min(max(int(playtime // 60), 1), 100)

        # Ajuste por popularidad
        factor_pop = popularidad_normalizada.get(appid, 1.0)

        # Ajuste por recencia (si ha jugado en las últimas 2 semanas)
        # Normalizamos el tiempo a horas (máx 15h = factor 1.5)
        horas_2w = playtime_2w / 60
        factor_reciente = 1.0 + min(horas_2w / 30, 0.5)  # entre 1.0 y 1.5

        # Peso final ajustado
        peso = int(base_peso * factor_pop * factor_reciente)
        juegos.extend([appid] * max(peso, 1))

    if len(juegos) >= 2:
        user_sequences_weighted.append(juegos)

print(f"📜 Secuencias ponderadas creadas: {len(user_sequences_weighted)}")
print("🎮 Ejemplo:", user_sequences_weighted[0][:100])


📜 Secuencias ponderadas creadas: 1643
🎮 Ejemplo: ['730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730', '730']


In [207]:
from gensim.models import Word2Vec

# Entrenamos el modelo Word2Vec
model = Word2Vec(
    sentences=user_sequences_weighted,
    vector_size=100,     # Dimensión del vector de embedding
    window=5,            # Tamaño del contexto
    min_count=1,         # Mínimo de apariciones para ser considerado
    workers=4,           # Núcleos para paralelizar
    sg=1,                # Skip-gram (1) o CBOW (0)
    epochs=200            # Número de épocas
)

print("✅ Entrenamiento del modelo Word2Vec completado")
model.save("word2vec_steam.model")


✅ Entrenamiento del modelo Word2Vec completado


In [208]:
def recomendar_juegos_word2vec_con_nombres(modelo, steam_id, api_key, topn=30, min_popularidad=0.05):
    import requests
    import pandas as pd

    # Cargar nombres desde CSV
    appid_to_name = pd.read_csv("nombres_juegos.csv").set_index("appid")["name"].to_dict()

    # Llamada a la API de Steam
    url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={api_key}&steamid={steam_id}&include_appinfo=1&format=json"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        juegos_usuario = data.get("response", {}).get("games", [])
        appids = [str(j["appid"]) for j in juegos_usuario if j.get("playtime_forever", 0) > 0]

        if not appids:
            return [], [], "⚠️ El usuario no tiene juegos con tiempo jugado."

    except Exception as e:
        return [], [], f"❌ Error al obtener juegos del usuario: {e}"

    # Filtrar appids válidos
    appids_validos = [a for a in appids if a in modelo.wv.key_to_index]
    ignorados = len(appids) - len(appids_validos)

    jugados_nombres = [appid_to_name.get(int(a), f"Unknown ({a})") for a in appids_validos]

    if not appids_validos:
        return jugados_nombres, [], "⚠️ Ningún juego jugado está en el vocabulario del modelo."

    # Recomendaciones
    try:
        recomendaciones = modelo.wv.most_similar(appids_validos, topn=topn * 2)  # Pedimos más para poder filtrar luego
        recomendados = []
        for appid, similitud in recomendaciones:
            frecuencia = modelo.wv.get_vecattr(appid, "count")
            porcentaje = (frecuencia / modelo.corpus_total_words) * 100
            if porcentaje < min_popularidad:
                continue  # Ignoramos juegos con poca popularidad
            nombre = appid_to_name.get(int(appid), f"Unknown ({appid})")
            recomendados.append({
                "appid": int(appid),
                "nombre": nombre,
                "score_similitud": round(similitud, 4),
                "popularidad_%": round(porcentaje, 4)
            })
            if len(recomendados) >= topn:
                break

    except Exception as e:
        return jugados_nombres, [], f"❌ Error al generar recomendaciones: {e}"

    mensaje = "✅ Recomendaciones generadas correctamente"
    if ignorados > 0:
        mensaje += f" (⚠️ {ignorados} juegos ignorados por no estar en el modelo)"

    return jugados_nombres, recomendados, mensaje


In [209]:
jugados, recomendados, estado = recomendar_juegos_word2vec_con_nombres(model, "76561198235790210", "F8A4003EBB12D2357E82A7D7ED29F694")

print("🎮 Juegos jugados:")
for nombre in jugados:
    print(f"- {nombre}")

print("\n✨ Recomendaciones:")
for reco in recomendados:
    print(f"- {reco['nombre']} (ID: {reco['appid']}) | Similitud: {reco['score_similitud']:.4f} | Popularidad: {reco['popularidad_%']:.2f}%")

print("\n" + estado)


🎮 Juegos jugados:

✨ Recomendaciones:

❌ Error al obtener juegos del usuario: 429 Client Error: Too Many Requests for url: http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=F8A4003EBB12D2357E82A7D7ED29F694&steamid=76561198235790210&include_appinfo=1&format=json


In [210]:
# import pandas as pd

# # Cargar el archivo CSV
# df = pd.read_csv("usuarios_steam_detallado.csv")

# # Solicitar el Steam ID
# steam_id = input("Introduce el Steam ID del usuario: ").strip()

# # Filtrar juegos jugados por ese usuario (playtime_forever > 0)
# juegos_usuario = df[(df["steam_id"] == int(steam_id)) & (df["playtime_forever"] > 0)]

# # Mostrar resultados
# if juegos_usuario.empty:
#     print("⚠️ Este usuario no tiene juegos con tiempo jugado o no existe en el archivo.")
# else:
#     print(f"\n🎮 Juegos jugados por el usuario {steam_id}:\n")
#     for _, fila in juegos_usuario.iterrows():
#         nombre = fila["name"]
#         tiempo = fila["playtime_forever"] // 60  # Pasar a horas aprox.
#         print(f"- {nombre} ({tiempo} h)")

#     print(f"\n🧾 Total de juegos jugados: {len(juegos_usuario)}")
